In [ ]:
import numpy as np
import math
import sympy as sp
from gurobipy import *
from scipy.optimize import minimize
from scipy.optimize import LinearConstraint
import scipy.optimize
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
# import networkx as nx
import timeit
from time import time
import numpy as np
import ctypes

# Supply and demand input

In [ ]:
t_coef = {5:2, 
          6:2, 
          7:2}
c_coef = {5:4, 
          6:4, 
          7:4}

utility = 20
h_bound = 2
M = 10**5


MODoper = [5,6,7]
MODoper_node = dict()
MODoper_node[5] = [25,26,27,28,29,30,31,32,33,34,35,36]
MODoper_node[6] = [37,38,39,40,41,42,43,44,45,46]
MODoper_node[7] = [47,48,49,50,51,52,53]
MODoper_node_ori = MODoper_node.copy()

V_coeff = {5:5,
           6:2,
           7:8}
V_coef = []
for ooper in V_coeff:
    for i in range(len(MODoper_node[ooper])):
        V_coef.append(V_coeff[ooper])


MODnode = [25,26,27,28,29,30,31,32,33,34,35,36,
          37,38,39,40,41,42,43,44,45,46,
          47,48,49,50,51,52,53]
MODnode_ori = [25,26,27,28,29,30,31,32,33,34,35,36,
          37,38,39,40,41,42,43,44,45,46,
          47,48,49,50,51,52,53]

MODnode_oper = {}
for operr in MODoper:
    for nodee in MODoper_node[operr]:
        MODnode_oper[nodee] = operr

MODnode_h = []
V_coef_h = dict()
for i in MODnode:
    for k in range(1,h_bound+1):
        MODnode_h.append(i*10+k)
        if k == 0:
            V_coef_h[i*10+k] = 0
        else:
            V_coef_h[i*10+k] = V_coef[MODnode.index(i)]
MODnode = MODnode_h
V_coef = V_coef_h
        
OD_set = [1,2,12,18,13,20]

In [ ]:
network_file="network_SiouxFalls.csv"
demand_file="demand_SiouxFalls_FULL.csv"
MODnode_match="MODnodes_SiouxFalls.csv"

df = pd.read_csv(network_file)
df = df.dropna()
# for iii in range(df.shape[0]):
#     if df['operator'][iii] == oper_subsidized:
#         a = df['operating cost'][iii]
#         df['operating cost'][iii] = a* (1-oper_sub_percentage)
MODnode_mat = pd.read_csv(MODnode_match)
MODnode_mat = MODnode_mat.dropna()

In [ ]:
df_fixed = df.loc[df['link type'] == 'Fixed']
df_fixed['fleet_size'] = 0
df_walk = df.loc[df['link type'] == 'walk']
df_walk['fleet_size'] = 0
df_fixed = pd.concat([df_fixed, df_walk], ignore_index=True)

df_mod_ori = df.loc[df['link type'] == 'MOD']
df_mod = df_mod_ori.copy()
for i in range(h_bound):
    df_mod['fleet_size'] = i+1
    df_mod['from_node'] = df_mod_ori['from_node']*10 + i+1
    df_mod['to_node'] = df_mod_ori['to_node']*10 + i+1
    df_fixed = pd.concat([df_fixed, df_mod], ignore_index=True)
    
df_trans = pd.DataFrame(columns=df_fixed.columns)
for j in range(h_bound):
    for i in range(MODnode_mat.shape[0]):
        data = [[0,MODnode_mat.iloc[i]['fixed_node'],MODnode_mat.iloc[i]['MOD_node']*10+j+1,0,0,0,MODnode_mat.iloc[i]['operator'],'Transfer to MOD',j+1]]
        df_trans_append = pd.DataFrame(data, columns=df_fixed.columns)
#         df_trans = df_trans.append(df_trans_append)
        df_trans = pd.concat([df_trans, df_trans_append], ignore_index=True)

        data = [[0,MODnode_mat.iloc[i]['MOD_node']*10+j+1,MODnode_mat.iloc[i]['fixed_node'],0,0,0,MODnode_mat.iloc[i]['operator'],'Transfer from MOD',j+1]]
        df_trans_append = pd.DataFrame(data, columns=df_fixed.columns)    
#         df_trans = df_trans.append(df_trans_append)
        df_trans = pd.concat([df_trans, df_trans_append], ignore_index=True)
df_all = pd.concat([df_fixed, df_trans], ignore_index=True)


#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# 2) Input demand file
df11 = pd.read_csv(demand_file)
df11_ = pd.DataFrame(columns = df11.columns)
for nodee in OD_set:
    df = df11.loc[df11['from_node'] == nodee]
#     df11_ = df11_.append(df)
    df11_ = pd.concat([df11_, df], ignore_index=True)
df11__ = pd.DataFrame(columns = df11.columns)
for nodee in OD_set:
    df = df11_.loc[df11['to_node'] == nodee]
    df11__ = pd.concat([df11__, df], ignore_index=True)
df11 = df11__.copy()
df11 = df11[['user_number','from_node','to_node','demand_amount','utility']]
df11 = df11.dropna()
df_=df11[df11.from_node != df11.to_node]
orig=list(df_["from_node"])
dest=list(df_["to_node"])
d=list(df_["demand_amount"])
for iii in range(len(d)):
    if d[iii]== 0:
        d[iii] = 100

df_dummy = pd.DataFrame(columns=df_fixed.columns)
for j in range(len(d)):
    data = [[0,orig[j],dest[j],utility,0,M,0,'Dummy link',0]]
    df_dummy_append = pd.DataFrame(data, columns=df_fixed.columns)
#     df_dummy = df_dummy.append(df_dummy_append)
    df_dummy = pd.concat([df_dummy, df_dummy_append], ignore_index=True)
# df_all = df_all.append(df_dummy)
df_all = pd.concat([df_all, df_dummy], ignore_index=True)


df_all = df_all.reset_index()
df_all = df_all.drop(columns = ['index'])
df_all['link number'] = list(range(1, df_all.shape[0]+1))

#1) Input the whole network
df2 = df_all.copy()
links=len(list(df2['from_node'])) # number of links
link={}
dummy_link = []
for i in range(0,links):
    new_result={'start':'','end':'','operating cost':'','capacity':'','travel cost':'','operator':'','fleet size':''}
    link[i]=new_result
    link[i]['start']=df2['from_node'][i]
    link[i]['end']=df2['to_node'][i]
    link[i]['travel cost']=df2['travel cost'][i]
    link[i]['operating cost']=df2['operating cost'][i]
    link[i]['capacity']=df2['capacity'][i]
    link[i]['operator']=df2['operator'][i]
    link[i]['fleet size']=df2['fleet_size'][i]
    if df2['link type'][i] == 'Dummy link':
        link[i]['travel cost']=utility
        dummy_link.append(i)
        
nodes=list(set().union(list(df2['to_node']), list(df2['from_node'])))
oper_list=set(list(df2['operator']))

demand=len(orig)
k={}
for i in nodes:
    k[i]=0 
    for s in range(demand):
        if i==dest[s]:
            k[i]-=d[s]
        if i==orig[s]:
            k[i]+=d[s]
#         if orig[s]==dest[s]:
#             k[i]=0
# print('k:',k)

MODoper_node_ori = MODoper_node.copy()

MODoper_node_h = dict()
for i in MODoper_node:
    MODoper_node_h[i] = []   
for i in range(h_bound):
    for nodee in MODoper_node:
        for k in MODoper_node[nodee]:
            MODoper_node_h[nodee].append(k*10+i+1)            
MODoper_node = MODoper_node_h.copy()


MODnode_oper_h = {}
for nodee in MODnode_oper:
    for i in range(h_bound):
        MODnode_oper_h[nodee*10+i+1] = MODnode_oper[nodee]
        
MODnode_oper = MODnode_oper_h.copy()

#------------------------------------------------------------------------------------------------------------------------------

#2) Input the FIXED network
df2 = df_all.loc[df_all['link type'] == 'Fixed']
# df2_ = df_all.loc[df_all['link type'] == 'walk']
# df2 = df2.append(df2_)
# df2 = df2.reset_index()
links_fixed=len(list(df2['from_node'])) # number of links
link_fixed={}
for i in range(0,links_fixed):
    new_result={'start':'','end':'','operating cost':'','capacity':'','travel cost':'','operator':'','fleet size':''}
    link_fixed[i]=new_result
    link_fixed[i]['start']=df2['from_node'][i]
    link_fixed[i]['end']=df2['to_node'][i]
    link_fixed[i]['travel cost']=df2['travel cost'][i]
    link_fixed[i]['operating cost']=df2['operating cost'][i]
    link_fixed[i]['capacity']=df2['capacity'][i]
    link_fixed[i]['operator']=df2['operator'][i]
    link_fixed[i]['fleet size']=df2['fleet_size'][i]
# print("link_fixed:",link_fixed)

#2) Input the transfer links
df2 = df_all.loc[df_all['link type'] == 'walk']
# df2 = df2.reset_index()
links_trans=len(list(df2['from_node'])) # number of links
link_trans={}
for i in list(df2.index):
    new_result={'start':'','end':'','operating cost':'','capacity':'','travel cost':'','operator':'','fleet size':''}
    link_trans[i]=new_result
    link_trans[i]['start']=df2['from_node'][i]
    link_trans[i]['end']=df2['to_node'][i]
    link_trans[i]['travel cost']=df2['travel cost'][i]
    link_trans[i]['operating cost']=df2['operating cost'][i]
    link_trans[i]['capacity']=df2['capacity'][i]
    link_trans[i]['operator']=df2['operator'][i]
    link_trans[i]['fleet size']=df2['fleet_size'][i]

#3) Mapping of the access links and exit links
df2 = df_all.loc[df_all['link type'] == 'Transfer to MOD']
# df2 = df2.reset_index()
df3 = df_all.loc[df_all['link type'] == 'MOD']
# df3 = df3.reset_index()
links_Acce=len(list(df2['from_node'])) # number of links
link_Acce={}
for i in list(df2.index):
    new_result={'start':'','end':'','operator':''}
    link_Acce[i]=new_result
    link_Acce[i]['start']=df2['from_node'][i]
    link_Acce[i]['end']=df2['to_node'][i]
    df4 = df3.loc[df3['from_node'] == link_Acce[i]['end']]
    df4 = df4.reset_index()
    link_Acce[i]['operator']=df4['operator'][0]
    link_Acce[i]['fleet size']=df4['fleet_size'][0]
# print("link_Acce:",link_Acce)

df2 = df_all.loc[df_all['link type'] == 'Transfer from MOD']
# df2 = df2.reset_index()
df3 = df_all.loc[df_all['link type'] == 'MOD']
# df3 = df3.reset_index()
links_Egre=len(list(df2['from_node'])) # number of links
link_Egre={}
for i in list(df2.index):
    new_result={'start':'','end':'','operator':''}
    link_Egre[i]=new_result
    link_Egre[i]['start']=df2['from_node'][i]
    link_Egre[i]['end']=df2['to_node'][i]
    df4 = df3.loc[df3['to_node'] == link_Egre[i]['start']]
    df4 = df4.reset_index()
    link_Egre[i]['operator']=df4['operator'][0]
    link_Egre[i]['fleet size']=df4['fleet_size'][0]
# print('link_Egre:',link_Egre)



#4) Mapping MOD operators to the access links
df2 = df_all.loc[df_all['link type'] == 'MOD']
df2 = df2.reset_index()
MOD_operator = list(np.unique(np.array(df2['operator'])))
df3 = df_all.loc[df_all['link type'] == 'Transfer to MOD']
MODoper_Acce = {}
for i in MOD_operator:
    MODoper_Acce[i] = []
for i in link_Acce:
    for oper in MOD_operator:
        if link_Acce[i]['operator'] == oper:
            MODoper_Acce[oper].append([link_Acce[i]['start'],link_Acce[i]['end']])

#5) MOD nodes
df2 = df_all.loc[df_all['link type'] == 'MOD']
from_nodes = np.array([df2['from_node']])
to_nodes = np.array([df2['to_node']])
MOD_nodes = np.hstack((from_nodes,to_nodes))
MOD_nodes = np.unique(MOD_nodes)
# print('MOD_nodes:',MOD_nodes)

#6) MOD LINKS
df2 = df_all.loc[df_all['link type'] == 'MOD']
# df2 = df2.reset_index()
links_MOD=len(list(df2['from_node'])) # number of links
link_MOD={}
for i in df2.index:
    new_result={'start':'','end':'','operating cost':'','capacity':'','travel cost':'','operator':'','fleet size':''}
    link_MOD[i]=new_result
    link_MOD[i]['start']=df2['from_node'][i]
    link_MOD[i]['end']=df2['to_node'][i]
    link_MOD[i]['travel cost']=df2['travel cost'][i]
    link_MOD[i]['operating cost']=df2['operating cost'][i]
    link_MOD[i]['capacity']=df2['capacity'][i]
    link_MOD[i]['operator']=df2['operator'][i]
    link_MOD[i]['fleet size']=df2['fleet_size'][i]
# print('link_MOD:',link_MOD)


link_from_to = dict()
for i in link:
    if i not in dummy_link:
        link_from_to[(link[i]['start'], link[i]['end'])] = i  
    

#--------------------------------------------------------------------------------------------------------------------------------------------
# Assignment matching problem
outflow=dict()
inflow=dict()
outflow_link=dict()
inflow_link=dict()
lst=[]
lst2=[]
for i in range(links):
    lst2.append((link[i]['start'],link[i]['end']))
    for s in range(demand):
        lst.append((link[i]['start'],link[i]['end'],s)) 
    a=link[i]['start']
    b=link[i]['end']
    if b in inflow:
        inflow[b].append(link[i]['start'])
        inflow_link[b].append(i)
    else:
        inflow[b]=[link[i]['start']]
        inflow_link[b]=[i]
    if a in outflow:
        outflow[a].append(link[i]['end'])
        outflow_link[a].append(i)
    else:
        outflow[a]=[link[i]['end']]
        outflow_link[a]=[i]
for i in nodes:
    if i not in inflow:
        inflow[i]={}
        inflow_link[i]=[]
    if i not in outflow:
        outflow[i]={}
        outflow_link[i]=[]
        
#--------------------------------------------------------------------------------------------------------------------------------------------

node = []
for i in outflow_link:
    node.append(i)

#--------------------------------------------------------------------------------------------------------------------------------------------

df1 = df_all.loc[df_all['link type'] == 'Transfer to MOD']
df2 = df_all.loc[df_all['link type'] == 'Transfer from MOD']
transin_oper = dict()
for i in range(len(link_fixed)+len(link_trans)+len(link_MOD),len(link_fixed)+len(link_trans)+len(link_MOD)+len(link_Acce)+len(link_Egre),2):
    transin_oper[i] = df1['operator'][i]
# print('transin_oper:',transin_oper)


oper_transin = dict()
for j in range(min(MODoper),max(MODoper)+1):
    oper_transin[j] = []
for i in range(len(link_fixed)+len(link_trans)+len(link_MOD),len(link_fixed)+len(link_trans)+len(link_MOD)+len(link_Acce)+len(link_Egre),2):
    oper = df1['operator'][i]
    for j in range(min(MODoper),max(MODoper)+1):
        if oper == j:
            heiheihei = oper_transin[j]
            heiheihei.append(i)
            oper_transin[j] = heiheihei
# print('oper_transin:',oper_transin)

#--------------------------------------------------------------------------------------------------------------------------------------------


kkk={}
for s in range(len(d)):
    for i in nodes:
        kkk[i,s]=0
        if i==dest[s]:
            kkk[i,s]=-d[s]
        if i==orig[s]:
            kkk[i,s]=d[s]
        if orig[s]==dest[s]:
            kkk[i,s]=0
            
def bbb_in_aaa(aaa,bbb):
    result = False
    if len(aaa) >= len(bbb):
        for index, value in enumerate(aaa):
            if value == bbb[0]:
                result = True
                for i in range(1, len(bbb)):
                    if aaa[index+i] != bbb[i]:
                        result = False
    return result

def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3



In [ ]:
# link_num = len(link)

oper_cost = []
for i in range(len(link_fixed)):
    oper_cost.append(link_fixed[i]['operating cost'])
    
cap = []
for i in range(len(link_fixed)):
    cap.append(link_fixed[i]['capacity'])
    
b = np.zeros((len(orig),len(link_fixed)))
for k in range(len(orig)):
    for links in range(len(link_fixed)):
        b[k,links] = min(cap[links],d[k])
        
a_alllink = list(link.keys())
a_fixedlink = list(link_fixed.keys())
a_translink = list(link_trans.keys())
a_MODlink = list(link_MOD.keys())
a_transinlink = list(link_Acce.keys())
a_transoutlink = list(link_Egre.keys())

# Import packages

In [ ]:
from MaaS_game_OD_subscribe import b_and_b, rank_by_obj, network_branch, obj_yv, single_branch, Dijkstra_cpp, flow_yv_finding, subgrad_opt, FW, obj, Dijkstra, solve_for_Y_vbounded, find_alpha_der_0
from Cost_allo_sub_bundle import pre_process_intsol, cost_allo_prep, cost_allo, subsidy_stablize, cost_allo_sub


# Set parameters

In [ ]:
num_iter = 8000
tolerance = 0.05
PRINT = 0
FW_tol = 10**-2
FW_conver_times = 5

utility = [20]
utility = utility*len(d)

# Solve the MaaS assignment game

In [ ]:
%%time

flows_link_best, flows_path_best, y_best, v_best, obj_best, ttl_cost_best, subsidy_obj_best, dual_p_best, path_set_best, p_user_opt_best, u_user_opt_best, p_oper_opt_best, u_oper_opt_best, S_path_best = b_and_b(PRINT, num_iter, tolerance, FW_tol, FW_conver_times, 
                                                                                             t_coef, c_coef, d, orig, dest, cap, b, oper_cost, V_coef, utility, 
                                                                                             link, link_fixed, link_trans, link_Acce, link_MOD, link_Egre, link_from_to,
                                                                                             a_alllink, a_MODlink, a_fixedlink, a_translink, 
                                                                                             transin_oper, MOD_nodes, nodes, MODnode_oper, 
                                                                                             MODnode_ori, MODoper_node_ori, MODnode, MODoper, MODoper_Acce,
                                                                                             outflow_link, inflow_link, dummy_link, oper_list)
    